In [1]:
import itertools
import os

import jax
import jax.numpy as jnp
import numpy as np
from numpy.polynomial.hermite import hermgauss

from vibrojet.basis_utils import ContrBasis, HermiteBasis
from vibrojet.keo import Gmat, com, pseudo
from vibrojet.potentials import h2co_AYTY
from vibrojet.taylor import deriv_list

jax.config.update("jax_enable_x64", True)

Equilibrium coordinates, around which the Taylor series expansions will be build

In [2]:
r0 = np.array([1.20337419, 1.10377465, 1.10377465, 2.1265833, 2.1265833, np.pi])

Define internal to Cartesian coordinate transformation

In [3]:
masses = [12.0, 15.99491463, 1.00782505, 1.00782505]  # masses of C, O, H, H

ncoo = len(r0)


# @eckart(r0, masses)
@com(masses)
def internal_to_cartesian(internal_coords):
    rCO, rCH1, rCH2, aOCH1, aOCH2, tau = internal_coords
    xyz = [
        [0.0, 0.0, 0.0],
        [0.0, 0.0, rCO],
        [
            rCH1 * jnp.sin(aOCH1) * jnp.cos(tau * 0.5),
            -rCH1 * jnp.sin(aOCH1) * jnp.sin(tau * 0.5),
            rCH1 * jnp.cos(aOCH1),
        ],
        [
            rCH2 * jnp.sin(aOCH2) * jnp.cos(tau * 0.5),
            rCH2 * jnp.sin(aOCH2) * jnp.sin(tau * 0.5),
            rCH2 * jnp.cos(aOCH2),
        ],
    ]
    return jnp.array(xyz, dtype=jnp.float64)

Generate expansions for the KEO and PES

In [4]:
gmat_max_order = 4  # max total expansion order

gmat_terms = np.array(
    [
        elem
        for elem in itertools.product(
            *[range(0, gmat_max_order + 1) for _ in range(len(r0))]
        )
        if sum(elem) <= gmat_max_order
    ]
)
pseudo_terms = gmat_terms
print("max expansion order for G-matrix:", gmat_max_order)
print("number of expansion terms in G-matrix:", len(gmat_terms))

# expansion of G-matrix

gmat_file = f"_h2co_gmat_coefs_{gmat_max_order}.npy"
if os.path.exists(gmat_file):
    print(
        f"load G-matrix expansion coefs from file {gmat_file} (delete file to recompute coefficients)"
    )
    gmat_coefs = np.load(gmat_file)
else:
    gmat_coefs = deriv_list(
        lambda x: Gmat(x, masses, internal_to_cartesian), gmat_terms, r0, if_taylor=True
    )
    np.save(gmat_file, gmat_coefs)

# expansion of pseudopotential

pseudo_file = f"_h2co_pseudo_coefs_{gmat_max_order}.npy"
if os.path.exists(pseudo_file):
    print(
        f"load pseudopotential expansion coefs from file {pseudo_file} (delete file to recompute coefficients)"
    )
    pseudo_coefs = np.load(pseudo_file)
else:
    pseudo_coefs = deriv_list(
        lambda x: pseudo(x, masses, internal_to_cartesian),
        gmat_terms,
        r0,
        if_taylor=True,
    )
    np.save(pseudo_file, pseudo_coefs)

max expansion order for G-matrix: 4
number of expansion terms in G-matrix: 210
load G-matrix expansion coefs from file _h2co_gmat_coefs_4.npy (delete file to recompute coefficients)
load pseudopotential expansion coefs from file _h2co_pseudo_coefs_4.npy (delete file to recompute coefficients)


In [5]:
poten_max_order = 6

poten_terms = np.array(
    [
        elem
        for elem in itertools.product(
            *[range(0, poten_max_order + 1) for _ in range(len(r0))]
        )
        if sum(elem) <= poten_max_order
    ]
)
print("max expansion order for PES:", poten_max_order)
print("number of expansion terms in PES:", len(poten_terms))

poten_file = f"_h2co_poten_coefs_{poten_max_order}.npy"
if os.path.exists(poten_file):
    print(
        f"load potential expansion coefs from file {poten_file} (delete file to recompute coefficients)"
    )
    poten_coefs = np.load(poten_file)
else:
    poten_coefs = deriv_list(h2co_AYTY.poten, poten_terms, r0, if_taylor=True)
    np.save(poten_file, poten_coefs)

max expansion order for PES: 6
number of expansion terms in PES: 924
load potential expansion coefs from file _h2co_poten_coefs_6.npy (delete file to recompute coefficients)


Define linear mapping between coordinates of Hermite functions and internal coordinates

In [22]:
mask = gmat_terms != 0
ind0 = np.where(mask.sum(axis=1) == 0)[0][0]
mu = np.diag(gmat_coefs[ind0])[:ncoo]

mask = poten_terms != 0
ind2 = np.array(
    [
        np.where((mask.sum(axis=1) == 1) & (poten_terms[:, icoo] == 2))[0][0]
        for icoo in range(ncoo)
    ]
)
freq = poten_coefs[ind2] * 2

lin_a = np.sqrt(np.sqrt(mu / freq))
lin_b = r0

print("x->r linear mapping parameters 'a':", lin_a)
print("x->r linear mapping parameters 'b':", lin_b)

# x->r linear mapping function
x_to_r_map = lambda x, icoo: lin_a[icoo] * x + lin_b[icoo]

x->r linear mapping parameters 'a': [0.05265804 0.11094744 0.11094744 0.15432971 0.15432971 0.30023422]
x->r linear mapping parameters 'b': [1.20337419 1.10377465 1.10377465 2.1265833  2.1265833  3.14159265]


In [23]:
x, w = hermgauss(60)
r = x_to_r_map(x, 5)
print(r * 180 / np.pi)

[  5.24143941  16.21994982  25.31138578  33.42785598  40.91734058
  47.9590202   54.66034877  61.09214917  67.30423167  73.33333905
  79.20758838  84.94913598  90.57586679  96.10251385 101.54142755
 106.90312083 112.19666571 117.42998853 122.61009416 127.7432395
 132.83507    137.89072877 142.91494519 147.91210793 152.88632609
 157.84148112 162.78127185 167.70925403 172.62887596 177.54351112
 182.45648888 187.37112404 192.29074597 197.21872815 202.15851888
 207.11367391 212.08789207 217.08505481 222.10927123 227.16493
 232.2567605  237.38990584 242.57001147 247.80333429 253.09687917
 258.45857245 263.89748615 269.42413321 275.05086402 280.79241162
 286.66666095 292.69576833 298.90785083 305.33965123 312.0409798
 319.08265942 326.57214402 334.68861422 343.78005018 354.75856059]


Define primitive Hermite basis

In [24]:
ncoo = len(r0)
nbas = [40] * ncoo
npoints = [60] * ncoo

p_r1, p_r2, p_r3, p_alpha1, p_alpha2, p_tau = [
    HermiteBasis(
        icoo,
        nbas[icoo],
        npoints[icoo],
        lambda x: x_to_r_map(x, icoo),
        lambda r: r - r0[icoo],
        lambda r: r - r0[icoo],
        gmat_terms[:, icoo],
        poten_terms[:, icoo],
    )
    for icoo in range(ncoo)
]

In [25]:
emax = [20000, 60000, 60000, 60000, 60000, 20000]
c_r1, c_r2, c_r3, c_alpha1, c_alpha2, c_tau = [
    ContrBasis(
        (icoo,),
        [p_r1, p_r2, p_r3, p_alpha1, p_alpha2, p_tau],
        lambda _: True,
        gmat_terms,
        gmat_coefs,
        poten_terms,
        poten_coefs,
        emax=emax[icoo],
    )
    for icoo in range(ncoo)
]

e = c_r1.enr
print("Solutions for r1 mode:\n", f"zpe = {e[0]}\n", e - e[0])
e = c_r2.enr
print("\nSolutions for r2 mode:\n", f"zpe = {e[0]}\n", e - e[0])
e = c_r3.enr
print("\nSolutions for r3 mode:\n", f"zpe = {e[0]}\n", e - e[0])
e = c_alpha1.enr
print("Solutions for alpha1 mode:\n", f"zpe = {e[0]}\n", e - e[0])
e = c_alpha2.enr
print("Solutions for alpha2 mode:\n", f"zpe = {e[0]}\n", e - e[0])
e = c_tau.enr
print("Solutions for tau mode:\n", f"zpe = {e[0]}\n", e - e[0])

Solutions for r1 mode:
 zpe = 3483.6927559110927
 [    0.          1765.27547405  3507.40314087  5226.49187839
  6922.76974337  8596.62232014 10248.6357592  11879.64246453
 13490.76562907 15083.45676233 16659.51853446 18221.10459613
 19770.6898851 ]

Solutions for r2 mode:
 zpe = 3740.4495793707006
 [    0.          2842.82311428  5577.85927463  8244.31092918
 10904.25731537 13618.17641528 16424.97968172 19342.60244015
 22376.86563032 25527.97855985 28793.77458816 32171.16489894
 35656.78470197 39247.28063583 42939.43561429 46730.22135124
 50616.82865708 54596.69263528 58667.53244527]

Solutions for r3 mode:
 zpe = 3740.4495793706997
 [    0.          2842.82311428  5577.85927463  8244.31092918
 10904.25731537 13618.17641528 16424.97968172 19342.60244015
 22376.86563032 25527.97855985 28793.77458816 32171.16489894
 35656.78470197 39247.28063583 42939.43561429 46730.22135124
 50616.82865708 54596.69263528 58667.53244527]
Solutions for alpha1 mode:
 zpe = 3419.781207227418
 [    0.      

In [26]:
c_r2_r3 = ContrBasis(
    (1, 2),
    [c_r1, c_r2, c_r3, c_alpha1, c_alpha2, c_tau],
    lambda _: True,
    gmat_terms,
    gmat_coefs,
    poten_terms,
    poten_coefs,
    emax=20000,
)

e = c_r2_r3.enr
print(len(e))
print("Solutions for r2+r3 modes:\n", f"zpe = {e[0]}\n", e - e[0])

36
Solutions for r2+r3 modes:
 zpe = 4444.493288948901
 [    0.          2825.50801119  2868.25536019  5570.57073722
  5587.29561921  5711.65639325  8245.48621281  8250.52713442
  8410.0695873   8486.07039932 10906.36125018 10908.37904348
 11078.08621035 11126.21098375 11229.24307216 13612.41040624
 13614.47340166 13739.63319222 13776.79531468 13858.87458617
 13941.25567198 16370.05123268 16386.67786219 16436.91322465
 16485.39315793 16539.26878668 16594.16639695 16647.48768492
 19061.04780205 19146.22362543 19215.10909151 19272.68532529
 19325.4977921  19354.34563678 19419.15901601 19421.81998357]


In [27]:
c_alpha1_alpha2 = ContrBasis(
    (3, 4),
    [c_r1, c_r2, c_r3, c_alpha1, c_alpha2, c_tau],
    lambda _: True,
    gmat_terms,
    gmat_coefs,
    poten_terms,
    poten_coefs,
    emax=20000,
)

e = c_alpha1_alpha2.enr
print(len(e))
print("Solutions for alpha1+alpha2 modes:\n", f"zpe = {e[0]}\n", e - e[0])

114
Solutions for alpha1+alpha2 modes:
 zpe = 3674.3337387464044
 [    0.          1265.58748343  1581.90790143  2534.58011322
  2843.75156213  3164.51359183  3807.72370557  4109.40087904
  4422.4520763   4747.29498495  5085.73111723  5379.63971414
  5684.58438613  6001.18924374  6329.7082676   6369.27867078
  6655.21278761  6951.73228495  7259.65248017  7579.44344281
  7659.00363651  7911.18309056  7936.82120593  8224.67172422
  8523.54381924  8834.10867403  8955.50269253  9156.67082741
  9225.11950245  9491.1168999   9504.12765102  9793.66937009
 10094.59957387 10259.33124914 10407.43409565 10520.71406694
 10732.29816529 10790.77085648 11068.86814808 11070.77648013
 11361.75099062 11571.00350458 11664.40384725 11824.16277662
 11979.08272331 12034.89888895 12085.21665249 12305.71666431
 12355.55063112 12636.33084956 12643.74803297 12890.99309225
 12928.44200973 13135.97560878 13232.43549741 13388.02507398
 13548.47568949 13648.61462789 13876.27447027 13919.03730924
 14200.33495032 142

In [28]:
e_max = 12000

e1 = np.copy(c_r1.enr - c_r1.enr[0])
e23 = np.copy(c_r2_r3.enr - c_r2_r3.enr[0])
e45 = np.copy(c_alpha1_alpha2.enr - c_alpha1_alpha2.enr[0])
e6 = np.copy(c_tau.enr - c_tau.enr[0])


def select_quanta(ind):
    i1, i23, i45, i6 = ind
    cond = e1[i1] + e23[i23] + e45[i45] + e6[i6] <= e_max
    return cond


sol = ContrBasis(
    (0, 1, 2, 3),
    [c_r1, c_r2_r3, c_alpha1_alpha2, c_tau],
    select_quanta,
    gmat_terms,
    gmat_coefs,
    poten_terms,
    poten_coefs,
    emax=20000,
    store_me=False,
)

e = sol.enr
print("Solutions:\n", f"zpe = {e[0]}\n", e - e[0])

Solutions:
 zpe = 5801.481866567859
 [    0.          1175.10683215  1255.24139737 ... 12756.13531386
 13165.14611739 13168.8743292 ]
